Program 1

Christian Basso, Patrick McDonald, David Litton

CSC 3310

January 30, 2024

In [14]:
import math

# Program 1

This code will take in a list of tuples that represent a point with an x and y coordinate. It will create the largest convex hull possible from the points and return the list of all the points that represent the hull.

Here are some array examples:

In [18]:
arr1 = ((0,0), (1, 1), (2, 2), (3, 3), (1, 4), (4, 5), 
        (5, 6), (6, 7), (7, 8), (8, 9), (0, 5), (9, 0), 
        (4, 9), (0, 6), (6, 9), (0, 7), (7, 9), (0, 8), 
        (8, 9), (0, 9), (7, 9), (6, 2))

arr2 = ((15, 5), (8, 29), (4, 22), (26, 19), (10, 5), 
        (21, 7), (19, 7), (30, 25), (14, 28), (19, 12))

arr3 = ((8, 16), (22, 7), (23, 27), (14, 16), (9, 20), 
        (29, 30), (10, 17), (20, 21), (4, 12), (29, 5))

arr4 = ((0, 0), (1, 1), (2, 2), (3, 3)) #This one shouldnt work...

arr5 = ((0, 0), (1, 1)) #This one shouldnt work...

arr6 = ((0, 0), (0, 1), (1, 0), (5, 5))

### Functions

In [22]:
'''
Function to calculate distance between two points
:author: Christian B
'''
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

'''
find_center_point(arr) function
returns the center point  as a tuple (x,y)
:param: arr - collection comprised of a series of x,y tuple points
:author: David L
'''
def find_center_point(arr):
    x_points = 0
    y_points = 0
    elements = 0
    for point in arr:
        x_points += point[0]
        y_points += point[1]
        elements += 1
    avg_x = x_points/elements
    avg_y = y_points/elements
    return (avg_x, avg_y)

'''
find_farthest_point(arr, center) function
returns the farthest point in the input arr from the input center
:param: arr - collection comprised of a series of x,y tuple points
:param: center - an x and y coordinate
:author: David L
'''
def find_farthest_point(arr, center):
    # define distance list
    distances = list()
    # calculate distance from center to all points and append to distance list
    for point in arr:
        distances.append(distance(point, center))
        
    # find max distance
    distances.sort(reverse=True)
    farthest_distance = distances[0]
    
    # find index of max distance
    distances.clear()
    for point in arr:
        distances.append(distance(point, center))
    index = distances.index(farthest_distance)
    
    # find and return point
    return arr[index]

#(Christian)
# define function to find most obtuse angle between current point and all other points
    # for each point
        # cacluate angle between current point and all other points
        # Return most obtuse point
'''
Calculate the angle 3 points make.
For the first itteration of the loop the points will be
The absolute center point (calculated), the farthest point, and the first point of intrest.
:author: Christian B
'''
def get_angle(A, B, C):
    # Calculate the lengths of the sides of the triangle
    AB = distance(A, B)
    BC = distance(B, C)
    AC = distance(A, C)
    # Use the law of cosines to calculate the angle at B
    cos_angle_B = (AB**2 + BC**2 - AC**2) / (2 * AB * BC)
    angle_B_radians = math.acos(cos_angle_B)
    angle_B_degrees = math.degrees(angle_B_radians)
    return angle_B_degrees

### Hull_algorithim
In the following cell our hull algorithim will be defined. This algorithm will take in a array **input_arr** and return/print an array of all points on the hull.

In [19]:
input_arr = arr6

#(David)
# def hull_algorithim (list of points)     
center_point = find_center_point(input_arr)
# Find the furthest point
    # Calculate center average of all points
    # Take point with longest line from the center
farthest_point = find_farthest_point(input_arr)
# Set furthest point as current point
current_point = farthest_point


# (Patrick)
# for each point
    # Find most obtuse point
    # Add current point to list of hull points
    # Make most obtuse angle point the current point
# return hull points

In [20]:
get_angle((0,0), (0,1), (1,1))

90.00000000000001